<h1>Se lee y preprocesa el dataset
<h3>Tambien, se identifican los nodos

In [3]:
from ast import literal_eval
from code import interact
from typing import List, Dict, Set, Tuple

import numpy as np
import pandas as pd
from ipykernel.pickleutil import interactive

df = pd.read_csv('ReformaPensional.csv')


def preprocess_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocesses a DataFrame by filling missing values in specific columns with default values.

    Columns processed:
        - 'geo': Fills missing values with "{}".
        - 'entities': Fills missing values with "{}".
        - 'in_reply_to_user_id': Fills missing values with 0.
        - 'ref_id': Fills missing values with 0.
        - 'ref_type': Fills missing values with "tweeted".
        - 'ref_author_id': Fills missing values with 0.
        - 'ref_author': Fills missing values with an empty string "".
        - 'ref_text': Fills missing values with an empty string "".
        - 'ref_note_tweet': Fills missing values with an empty string "".

    Parameters:
        df (pd.DataFrame): Input DataFrame containing the data to process.

    Returns:
        pd.DataFrame: Processed DataFrame with missing values replaced.

    Raises:
        ValueError: If one or more expected columns are missing from the DataFrame.
    """

    default_values = {
        "text": "",
        "created_at": 0,
        "public_metrics": "{}",
        'entities': "{}",
        'author_username': "",
        'ref_type': "tweeted",
        'ref_author': "",
        'ref_text': "",
        'ref_note_tweet': ""
    }
    missing_columns = [col for col in default_values if col not in df.columns]
    if missing_columns:
        raise ValueError(
            f"El Dataset no contiene las siguientes columnas necesarias: {', '.join(missing_columns)}"
        )
    df = df[["text", "created_at", "public_metrics", "entities", "author_username", "ref_type",
             "ref_author", "ref_text", "ref_note_tweet"]]
    for col, default in default_values.items():
        df[col] = df[col].fillna(default)
    return df


def get_mentions_list(tweet_entities: str) -> List[str]:
    """
    Extracts the list of mentioned usernames from a tweet's entity data.

    Parameters:
        tweet_entities (str): JSON string representing the tweet's entities.

    Returns:
        List[str]: List of mentioned usernames in the tweet.
    """
    try:
        return [mention['username'] for mention in literal_eval(tweet_entities).get("mentions", [])]
    except:
        raise ValueError(f"Invalid tweet_entities format")


def identify_nodes(df: pd.DataFrame) -> List[str]:
    """
    Identifies unique nodes in the DataFrame based on tweet authors and mentions.

    Parameters:
        df (pd.DataFrame): DataFrame containing tweet data with the following columns:
            - 'author_username': The username of the tweet's author.
            - 'ref_author': The username of the referenced author.
            - 'entities': A JSON string containing tweet entities.

    Returns:
        List[str]: A list of unique usernames representing the nodes.

    """
    users: Set[str] = set(np.concatenate((
        df['author_username'].dropna().unique(),
        df['ref_author'].dropna().unique()
    )))

    for tweet_entities in df['entities'].dropna():
        try:
            tweet_mentions = get_mentions_list(tweet_entities)
            users.update(tweet_mentions)
        except ValueError:
            continue

    users.discard("")

    return list(users)


df = preprocess_dataframe(df)
users = identify_nodes(df)
df

C:\Users\jujov\AppData\Local\Temp\ipykernel_20584\2840783449.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(default)


,text,created_at,public_metrics,entities,author_username,ref_type,ref_author,ref_text,ref_note_tweet
0,@petrogustavo Ahora son los ancianos la CORTIN...,2024-06-22 23:59:46+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 0, 'end': 13, 'usernam...",JULIOTORRESB3,replied_to,petrogustavo,No entiendo la inquina contra millones de anci...,No entiendo la inquina contra millones de anci...
1,RT @ThomasitaD: Se oponen a la reforma pension...,2024-06-22 23:59:42+00:00,"{'retweet_count': 37, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 14, 'usernam...",CURTlSEOso,retweeted,ThomasitaD,Se oponen a la reforma pensional que le ofrece...,
2,RT @petrogustavo: A llenar la plaza de Bolívar...,2024-06-22 23:59:40+00:00,"{'retweet_count': 3585, 'reply_count': 0, 'lik...","{'mentions': [{'start': 3, 'end': 16, 'usernam...",defrankns,retweeted,petrogustavo,A llenar la plaza de Bolívar con la tercera ed...,
3,RT @RobertoMTico: No era éste👇payaso de David ...,2024-06-22 23:59:31+00:00,"{'retweet_count': 248, 'reply_count': 0, 'like...","{'mentions': [{'start': 3, 'end': 16, 'usernam...",HectorA87524815,retweeted,RobertoMTico,"No era éste👇payaso de David Luna, junto con el...",
4,RT @willj84753699: ¡ATENCIÓN COLOMBIA!\nAmigos...,2024-06-22 23:59:22+00:00,"{'retweet_count': 292, 'reply_count': 0, 'like...","{'mentions': [{'start': 3, 'end': 17, 'usernam...",alperezv,retweeted,willj84753699,"¡ATENCIÓN COLOMBIA!\nAmigos, vamos con este ll...",
...,...,...,...,...,...,...,...,...,...
2483,Ya me llegó el comunicado de Colfondos … la Re...,2024-06-29 02:22:55+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'annotations': [{'start': 29, 'end': 37, 'pro...",azukittamami,tweeted,,,
2484,@LauraWillsO @CongresoVisible @CPolUniandes @f...,2024-06-29 02:08:05+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 0, 'end': 12, 'usernam...",Blochh16,replied_to,Blochh16,@LauraWillsO @CongresoVisible @CPolUniandes @f...,
2485,@FulanoZuluaga Apreciado Juan P. Aunque yo he ...,2024-06-29 02:06:56+00:00,"{'retweet_count': 0, 'reply_count': 2, 'like_c...","{'mentions': [{'start': 0, 'end': 14, 'usernam...",ESaenzRovner,replied_to,FulanoZuluaga,@ESaenzRovner Los fondos de ahorro pensional s...,
2486,1.El Guerrillero Presidente anuncia para el 20...,2024-06-29 02:05:19+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'urls': [{'start': 281, 'end': 304, 'url': 'h...",manuel_pascuas,tweeted,,,


<h1>Limpieza y procesamiento de las opiniones de cada usuario

In [5]:
import re
import emoji
from numpy import ndarray

user_to_index: Dict[str, int] = {user: idx for idx, user in enumerate(users)}
index_to_user: Dict[int, str] = {index: user for user, index in user_to_index.items()}


def clean_text(text: str) -> str:
    """
    Preprocesses a text by replacing URLs and emojis, and removing extra spaces.

    Parameters:
        text (str): Original text to preprocess.

    Returns:
        str: Preprocessed text.
    """
    text = re.sub(r'http\S+|www\S+', ':url:', text)  # Replace URLs with ':url:'
    text = emoji.demojize(text, language="es")  # Replace emojis with text descriptions
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and trim
    return text


def build_users_tweet_text(
        df: pd.DataFrame,
        user_to_index: Dict[str, int]
) -> ndarray[Set[str]]:
    """
    Builds a list of sets containing preprocessed opinions or interactions for each user.

    Params:
        df (pd.DataFrame): DataFrame containing tweet data.
        user_to_index (Dict[str, int]): Dictionary mapping each username to a unique index.

    Returns:
        ndarray: Array of sets with preprocessed opinions or interactions for each user.
    """
    n = len(user_to_index)
    users_tweet_text = [set() for _ in range(n)]

    for row in df.itertuples(index=False):
        author_idx = user_to_index[row.author_username]
        ref_author = row.ref_author if row.ref_author else None
        ref_text = row.ref_note_tweet if row.ref_note_tweet else row.ref_text

        if row.ref_type == "tweeted":
            users_tweet_text[author_idx].add(clean_text(row.text))
        elif row.ref_type == "retweeted" and ref_text:
            clean_ref_text = clean_text(ref_text)
            users_tweet_text[author_idx].add(clean_ref_text)

            if ref_author:
                ref_author_idx = user_to_index[ref_author]
                users_tweet_text[ref_author_idx].add(clean_ref_text)
        elif row.ref_type in ["quoted", "replied_to"] and ref_text:
            interaction_text = f'{clean_text(row.text)}\n[{row.ref_type} @{ref_author}: "{clean_text(ref_text)[:50]}"]'
            users_tweet_text[author_idx].add(interaction_text)

            if ref_author and ref_author in user_to_index:
                ref_author_idx = user_to_index[ref_author]
                users_tweet_text[ref_author_idx].add(clean_text(ref_text))
    return np.array(users_tweet_text, dtype=object)


users_tweet_text = build_users_tweet_text(df, user_to_index)
users_tweet_text

array([{'#ÚLTIMAHORA | Estalló escándalo con reforma (aprobada) del Gobierno; destapan reuniones y pagos que hubo :url: :url:', 'Dejen de criticar al gobierno del cambio por el mico en la reforma pensional que generosamente le redactó Asofondos. Entiendan que el acuerdo nacional incluye regalarle billones al gran capital con las cotizaciones del pueblo trabajador.'},
       {'alias Barbarita @lunadavid nuevamente en la palestra pública por sus actuaciones en contra del pueblo, además del desmonte del chisme de las chuzadas a Magistrados (Uribe lo sabe bien, es su costumbre), aquí la prueba del trabajo a favor de los banqueros con la reforma pensional! :url:\n[quoted @ALFREDOSAADEV: "La hipocresía en pasta. David luna metiendo un mic"]'},
       {'¿Les pesan los viejitos y por eso defienden a la señora del piqueteadero ó no les pesan y por eso se oponen a la reforma pensional? Decídanse, plaga de mierda.'},
       ...,
       {'La Reforma Pensional ya fue aprobada y con ella lograremos 

<h1>Se determina la postura
<h3>En este caso se las comparto calculadas, porque ese codigo es muy extenso y creo no viene al caso, aunque lo puedo compartir

In [6]:
from json import load

stances = load(open('testing_result.json'))
stances

{'porfiriohe': None,
 'ANIABELLO_R': None,
 'GustavoMoreno__': None,
 'DonDerecho__': None,
 'UNGRD': None,
 'PGN_COL': None,
 'PazLaboral': None,
 'oliviarueda1': None,
 'Albert135Carlos': None,
 'LauraWillsO': None,
 'estebanquincar': None,
 'kcv': None,
 'Chestereta': None,
 'PadreChuchoOf': None,
 'SebastianNohra': None,
 'ProCentrismo': None,
 'marthaperaltae': None,
 '_JAYAL_': None,
 'laurisarabia': None,
 'AlcaldiaCTG': None,
 'AELopezP': None,
 'Alcaldiakennedy': None,
 'FiscaliaCol': None,
 'Angela_OnAir': None,
 'WEIMAR_T': None,
 'Visi0naria': None,
 'MiguelPoloP': None,
 'Supersalud': None,
 'uni_cartagena': None,
 'VivianaVargasVi': None,
 'MaElviraSalazar': None,
 '_Robert_91': None,
 'canalcartagena': None,
 'PartidoASI_': None,
 'JotaPeHernandez': None,
 'DilianFrancisca': None,
 'facisouniandes': None,
 'ColombiaAcoset': None,
 'agmethescaf': None,
 'partidodelaucol': None,
 'JuanitaGomezL': None,
 'cabogadosa1': None,
 'OscuraColombia': None,
 'tentativadtrade': None

<h4>Funciones auxiliares, para normalizar y convertir textos a espacios vectoriales

In [8]:
from numpy import ndarray
from functools import cache



def normalization_min_max(matrix: ndarray) -> ndarray:
    """
    Applies log transformation followed by Min-Max normalization to a matrix.

    Parameters:
        matrix (ndarray): A numerical 2D array to be normalized.

    Returns:
        ndarray: A transformed and normalized matrix, where values are scaled to the range [0, 1].
    """
    matrix_transformed = np.log1p(matrix)

    # Min-Max normalization
    min_value = matrix_transformed.min()
    max_value = matrix_transformed.max()
    if max_value > min_value:
        matrix_transformed = (matrix_transformed - min_value) / (max_value - min_value)
    matrix_transformed = np.round(matrix_transformed, 3)
    return matrix_transformed


@cache
def get_embeddings(text: str, similarity_model):
    """
    Computes and caches the embeddings for a given opinion, removing additional information
    after a newline and within square brackets.

    Parameters:
        text (str): The opinion text to compute embeddings for.

    Returns:
        Embeddings of the cleaned opinion.
    """
    cleaned_text = re.sub(r'\n\[.*?\]', '', text).strip()
    return similarity_model.encode(cleaned_text, clean_up_tokenization_spaces=True, normalize_embeddings=True)


<h1>Heurística de interacciones
<h5>Necesaria para optimizar una de las que usted me pidió

In [14]:
def build_interaction_matrix(
        df: pd.DataFrame,
        user_to_index: Dict[str, int]
) -> Tuple[ndarray, ndarray, ndarray]:
    """
    Builds a mentions matrix quantifying interactions between users, along with a matrix of dates.

    Parameters:
        df (pd.DataFrame): DataFrame containing tweet data with the following columns:
            - 'author_username': The username of the tweet's author.
            - 'entities': A JSON string representing tweet entities.
            - 'ref_author': The username of the referenced author (if any).
            - 'created_at': Timestamp of the tweet's creation.
        user_to_index (Dict[str, int]): Dictionary mapping usernames to unique indices.

    Returns:
        Tuple[ndarray, ndarray, ndarray]:
            - Normalized mentions matrix (n x n), where [i, j] represents the interaction count from user j to user i.
            - Matrix of lists containing timestamps for mentions.
            - Rounded mentions matrix before normalization.
    """
    n = len(user_to_index)
    interactions_matrix = np.zeros((n, n), dtype=int)

    for row in df.itertuples(index=False):
        author = row.author_username
        author_idx = user_to_index.get(author)
        mentions = set(get_mentions_list(row.entities))

        if row.ref_author:
            mentions.add(row.ref_author)

        for mentioned_user in mentions:
            mentioned_user_idx = user_to_index.get(mentioned_user)

            if mentioned_user_idx is not None and author_idx is not None:
                interactions_matrix[mentioned_user_idx, author_idx] += 1

    normalized_matrix = normalization_min_max(interactions_matrix)
    rounded_matrix = np.round(interactions_matrix, 3)

    return normalized_matrix, rounded_matrix


interactions_matrix, interactions_matrix_nonNorm = build_interaction_matrix(df, user_to_index)
interactions_df = pd.DataFrame(interactions_matrix, index=users, columns=users)
interactions_df

,alhajasid,luzalean,Letrinas,JairoRicardoOs1,MglBTA13,jotapias,scarBon26585983,Alicia43121877,CarlosAguilarOr,Anitta__Alvarez,...,madyadams1,BlancaD24056009,sw33ttiny,pedromontoyapri,haroldmarulanda,juan93098,Juanfelipeus,angelimpulsor,DieFernando,nohoritahdeza
alhajasid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
luzalean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Letrinas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JairoRicardoOs1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MglBTA13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
juan93098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Juanfelipeus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
angelimpulsor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DieFernando,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h1>Heurística de acuerdo/desacuerdo
<h5>Genera una red no dirigida

In [11]:
import time
from numpy import ndarray

def build_agreement_matrix(
        mentions_matrix: ndarray,
        users_stances: Dict[str, float],
        index_user: Dict[int, str],
        agreement_threshold: float = 0.2
) -> ndarray:
    """
    Computes an agreement matrix based on users' stance similarities and mentions.

    Parameters:
        mentions_matrix (ndarray): Matrix indicating mentions between users (n x n).
        users_stances (Dict[str, float]): Dictionary mapping user identifiers to their stances.
        index_user (Dict[int, str]): Dictionary mapping indices to user identifiers.
        agreement_threshold (float): Threshold for determining agreement (default is 0.2).

    Returns:
        ndarray: Agreement matrix (n x n) where:
                 - 1 indicates agreement,
                 - -1 indicates disagreement,
                 - 0 indicates no interaction or invalid stances.
    """
    n = len(users_stances)
    users_agreement = np.zeros((n, n), dtype=float)

    for i in range(n - 1):
        user_i = index_user.get(i)
        stance_i = users_stances.get(user_i)
        if stance_i is None:
            continue

        for j in range(i + 1, n):
            user_j = index_user.get(j)
            stance_j = users_stances.get(user_j)
            if stance_j is None or (mentions_matrix[i, j] == 0 and mentions_matrix[j, i] == 0):
                continue

            agreement = abs(stance_i - stance_j) < agreement_threshold
            users_agreement[i, j] = 1 if agreement else -1

    return users_agreement


C:\Users\jujov\OneDrive\Escritorio\I\UNIVERSIDAD JUAN JOSE\semestre 10\TG2\influenceGraph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
agreement_matrix = build_agreement_matrix(interactions_matrix, stances, index_to_user)
agreement_DF = pd.DataFrame(agreement_matrix, index=users, columns=users)
agreement_DF

,alhajasid,luzalean,Letrinas,JairoRicardoOs1,MglBTA13,jotapias,scarBon26585983,Alicia43121877,CarlosAguilarOr,Anitta__Alvarez,...,madyadams1,BlancaD24056009,sw33ttiny,pedromontoyapri,haroldmarulanda,juan93098,Juanfelipeus,angelimpulsor,DieFernando,nohoritahdeza
alhajasid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
luzalean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Letrinas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JairoRicardoOs1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MglBTA13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
juan93098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Juanfelipeus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
angelimpulsor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DieFernando,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h1>Heurística de afinidades
<h5>Genera digrafos

In [ ]:
from sentence_transformers import SentenceTransformer
similarity_model = SentenceTransformer('jaimevera1107/all-MiniLM-L6-v2-similarity-es')


def build_affinities_matrix(
        users_tweet_text: List[Set[str]],
        users_stances: Dict[str, float],
        index_user: Dict[int, str],
        mentions_matrix_nonNorm: np.ndarray,
) -> np.ndarray:
    """
    Calculates the affinity between users based on similarity of opinions and polarities.

    Params:
        users_tweet_text (List[set[str]]): Preprocessed opinions for each user.
        users_stances (Dict[str, float]): Dictionary of user stances on a particular topic.
        index_user (Dict[str, int]): Mapping of user identifiers to their index.
        mentions_matrix_nonNorm (np.ndarray): Matrix of user interactions/mentions.

    Returns:
        np.ndarray: Affinity matrix between users.
    """
    print("working...")
    a = time.time()
    n = len(users_tweet_text)
    users_affinity = np.zeros((n, n), float)

    user_with_stances = {i: users_stances[user] for i, user in enumerate(index_user.values()) if
                         users_stances[user] is not None}

    def calculate_embeddings(index):
        opinions = users_tweet_text[index]
        embeddings = np.array([get_embeddings(opinion, similarity_model) for opinion in opinions])
        return embeddings

    embeddings = {}
    for i in range(n - 1):
        if i not in user_with_stances:
            continue
        stance_i = user_with_stances[i]
        embeddings_i = calculate_embeddings(i) if i not in embeddings else embeddings.pop(i)
        for j in range(i + 1, n):
            if j not in user_with_stances:
                continue
            stance_j = user_with_stances[j]
            embeddings_j = calculate_embeddings(j) if j not in embeddings else embeddings[j]
            embeddings[j] = embeddings_j
            stance_diff = abs(stance_i - stance_j)
            if stance_diff < 0.2:  #eliminar esta condición si se desea hacer un clique...
                similarity_opinions: float = similarity_model.similarity(embeddings_i,
                                                                         embeddings_j).mean()

                affinity_value_ij = similarity_opinions * mentions_matrix_nonNorm[i, j]
                affinity_value_ji = similarity_opinions * mentions_matrix_nonNorm[j, i]

                #se asigna la afinidad si la hay, o la similitud de sus opiniones si es alta
                users_affinity[i, j] = affinity_value_ij if affinity_value_ij != 0 else (
                    similarity_opinions if similarity_opinions > .7 else 0)
                users_affinity[j, i] = affinity_value_ji if affinity_value_ji != 0 else (
                    similarity_opinions if similarity_opinions > .7 else 0)
        b = time.time()
        print(f"affinity work: {i=}, {(i + 1) / (n - 1):.1%}, {b - a:.1f}s", end="\r")

    users_affinity = normalization_min_max(users_affinity)
    users_affinity[users_affinity < 0.01] = 0
    print("\naffinity work: done")
    return users_affinity

In [15]:
affinities_matrix = build_affinities_matrix(users_tweet_text, stances, index_to_user, interactions_matrix_nonNorm)
affinities_df = pd.DataFrame(affinities_matrix, index=users, columns=users)
affinities_df

working...
affinity work: i=2302, 100.0%, 231.9s
affinity work: done


,alhajasid,luzalean,Letrinas,JairoRicardoOs1,MglBTA13,jotapias,scarBon26585983,Alicia43121877,CarlosAguilarOr,Anitta__Alvarez,...,madyadams1,BlancaD24056009,sw33ttiny,pedromontoyapri,haroldmarulanda,juan93098,Juanfelipeus,angelimpulsor,DieFernando,nohoritahdeza
alhajasid,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.227,0.0,0.0,0.0,0.0
luzalean,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
Letrinas,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
JairoRicardoOs1,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
MglBTA13,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
juan93098,0.227,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
Juanfelipeus,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
angelimpulsor,0.000,0.0,0.0,0.0,0.0,0.0,0.295,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
DieFernando,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
